# Package Installation

In [3]:
!pip install mediapipe

In [2]:
import cv2
import mediapipe as mp
import time

## Show the Webcam

In [5]:
cap = cv2.VideoCapture(0)
ptime = 0

while True:
    success, img = cap.read()
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    ctime = time.time()
    fps = 1 / (ctime - ptime)
    ptime = ctime
    cv2.putText(img, f'FPS:{int(fps)}', (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.imshow("Test", img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyWindow("Test")

## Hand Anchor Class

In [3]:
class handDetector():
    def __init__(self, mode = False, maxHands = 2, detectionConf = 0.85, trackingConf = 0.85):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionConf = detectionConf
        self.trackingConf = trackingConf

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(static_image_mode = self.mode, 
                                        max_num_hands = self.maxHands,
                                        min_detection_confidence = self.detectionConf,
                                        min_tracking_confidence = self.trackingConf)
        # self.hands = self.mpHands.Hands(static_image_mode=False, max_num_hands=2, min_detection_confidence=0.5, min_tracking_confidence=0.5)
        self.mpDraw = mp.solutions.drawing_utils
        
    def findHands(self,img, draw = True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        # print(results.multi_hand_landmarks)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS)
        return img

    def findPosition(self, img, handNo = 0, draw = True):
        lmlist = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lmlist.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 7, (255, 0, 255), cv2.FILLED)
        return lmlist
    

def main():
    pTime = 0
    cTime = 0
    cap = cv2.VideoCapture(0)
    detector = handDetector()

    while True:
        success, img = cap.read()
        img = detector.findHands(img)
        lmlist = detector.findPosition(img)
        if len(lmlist) != 0:
            print(lmlist)

        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime

        cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3)

        cv2.imshow("Hand_Detector", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyWindow("Hand_Detector")


if __name__ == "__main__":
    main()

[[0, 94, 362], [1, 137, 365], [2, 174, 349], [3, 199, 325], [4, 211, 302], [5, 165, 302], [6, 193, 269], [7, 210, 251], [8, 224, 237], [9, 142, 286], [10, 168, 254], [11, 186, 239], [12, 201, 229], [13, 117, 278], [14, 138, 244], [15, 154, 224], [16, 169, 212], [17, 91, 277], [18, 101, 248], [19, 110, 231], [20, 120, 221]]
[[0, 130, 328], [1, 169, 330], [2, 205, 313], [3, 228, 288], [4, 239, 264], [5, 199, 265], [6, 223, 238], [7, 238, 223], [8, 251, 210], [9, 178, 250], [10, 200, 223], [11, 215, 211], [12, 228, 202], [13, 155, 242], [14, 173, 210], [15, 186, 191], [16, 200, 177], [17, 128, 240], [18, 136, 211], [19, 144, 194], [20, 154, 181]]
[[0, 146, 322], [1, 196, 309], [2, 236, 287], [3, 264, 264], [4, 271, 239], [5, 229, 253], [6, 252, 227], [7, 266, 209], [8, 279, 192], [9, 210, 238], [10, 232, 209], [11, 246, 192], [12, 260, 179], [13, 190, 228], [14, 206, 200], [15, 219, 181], [16, 231, 166], [17, 166, 224], [18, 174, 197], [19, 180, 179], [20, 186, 164]]
[[0, 525, 303], [1, 4

### Hand Anchor Class - Trial Run

In [4]:
import HandTrackingModule
HandTrackingModule.main()

NameError: name 'handDetector' is not defined

## Hand Anchor with Calculation

In [7]:
import cv2
import mediapipe as mp
import time

cap = cv2.VideoCapture(0)

mpHands = mp.solutions.hands
hands = mpHands.Hands(static_image_mode=False,
                      max_num_hands=2,
                      min_detection_confidence=0.85,
                      min_tracking_confidence=0.85)
mpDraw = mp.solutions.drawing_utils

pTime = 0
cTime = 0

while True:
    success, img = cap.read()
    h, w, c = img.shape
    imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    allHands = []
    results = hands.process(imgRGB)
    """
    if results.multi_hand_landmarks:
        print(type(results.multi_hand_landmarks))
        print(results.multi_hand_landmarks)    
    if results.multi_handedness:
        print(type(results.multi_handedness))
        print(results.multi_handedness)  
       
    if results.multi_hand_landmarks:
        x = zip(results.multi_handedness,results.multi_hand_landmarks)
        print(list(x))
        break
    """     
    
    if results.multi_hand_landmarks:    
        for handType,handLms in zip(results.multi_handedness,results.multi_hand_landmarks):
            myHand = {}  # a dict
            lmlist = []
            xlist = []
            ylist = []
            
            for id, lm in enumerate(handLms.landmark):
                px, py = int(lm.x *w), int(lm.y*h)
                cv2.circle(img, (px,py), 3, (255,0,255), cv2.FILLED)
                
                lmlist.append([px, py])
                xlist.append(px)
                ylist.append(py)
            
            # box boundary defined
            xmax, xmin = max(xlist), min(xlist)
            ymax, ymin = max(ylist), min(ylist)
            boxW, boxH = xmax - xmin, ymax - ymin
            bbox = xmin, boxW, ymin, boxH
            cx, cy = bbox[0] + (bbox[2] // 2), \
                         bbox[1] + (bbox[3] // 2)
            
            # define hand
            myHand["lmlist"] = lmlist
            myHand["bbox"] = bbox
            myHand["center"] = (cx, cy)
            if handType.classification[0].label == "Right":
                myHand["type"] = "Left"
            else:
                myHand["type"] = "Right"
                
            # print(type(myHand))
            allHands.append(myHand)
            
            # draw hand anchor
            mpDraw.draw_landmarks(img, handLms, mpHands.HAND_CONNECTIONS)
            # cv2.rectangle(img, (bbox[0] - 20, bbox[1] - 20), (bbox[0] + bbox[2] + 20, bbox[1] + bbox[3] + 20), (255, 0, 255), 2)
            cv2.rectangle(img, (xmin-10, ymin-10), (xmax+10, ymax+10), (255, 0, 255), 2)
            # cv2.putText(img, myHand["type"] + " org", (bbox[0], bbox[1]), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 255), 2)            
            cv2.putText(img, myHand["type"], (xmin - 15, ymin - 15), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 255), 2)
            
        
            tipIds = [4, 8, 12, 16, 20]
            fingers = []
            # Thumb
            if myHand["type"] == "Right":
                if lmlist[tipIds[0]][0] > lmlist[tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)
            else:
                if lmlist[tipIds[0]][0] < lmlist[tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)
                
            # 4 Fingers
            for id in range(1, 5):
                if lmlist[tipIds[id]][1] < lmlist[tipIds[id] - 2][1]:
                    fingers.append(1)
                else:
                    fingers.append(0)

            cv2.putText(img, str(fingers), (xmin - 15, ymax + 35), cv2.FONT_HERSHEY_PLAIN, 1, (0, 100, 255), 1)
    
    """  
    if (len(allHands) != 0):
        print("allHands: " + str(len(allHands)))
    """  
    
    
    cTime = time.time()
    fps = 1/(cTime-pTime)
    pTime = cTime
    cv2.putText(img,str(int(fps)), (10,70), cv2.FONT_HERSHEY_PLAIN, 3, (255,0,255), 3)

    cv2.imshow("Hand_Detector", img)    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cv2.destroyWindow("Hand_Detector")

## Hand Anchor With Calculation Class

In [9]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import cv2
import mediapipe as mp
import time


# In[2]:


class HandDetector():
    def __init__(self, mode = False, maxHands = 2, detectionConf = 0.85, trackingConf = 0.85):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionConf = detectionConf
        self.trackingConf = trackingConf

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(static_image_mode = self.mode, 
                                        max_num_hands = self.maxHands,
                                        min_detection_confidence = self.detectionConf,
                                        min_tracking_confidence = self.trackingConf)
        
        self.mpDraw = mp.solutions.drawing_utils
        
    def findHands(self,img, draw = True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        # print(results.multi_hand_landmarks)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS)
        return img

    def findPosition(self, img, handNo = 0, draw = True):

        lmlist = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lmlist.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 7, (255, 0, 255), cv2.FILLED)
        return lmlist


# In[3]:


def main():
    pTime = 0
    cTime = 0
    cap = cv2.VideoCapture(0)
    detector = HandDetector()

    while True:
        success, img = cap.read()
        img = detector.findHands(img)
        lmlist = detector.findPosition(img)
        if len(lmlist) != 0:
            print(lmlist[4])

        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime

        cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3)

        cv2.imshow("Hand_Detector", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyWindow("Hand_Detector")


if __name__ == "__main__":
    main()

[4, 147, 130]
[4, 149, 128]
[4, 152, 131]
[4, 149, 134]
[4, 150, 133]
[4, 147, 135]
[4, 149, 135]
[4, 145, 136]
[4, 142, 137]
[4, 143, 137]
[4, 139, 138]
[4, 137, 139]
[4, 137, 139]
[4, 134, 141]
[4, 132, 142]
[4, 132, 142]
[4, 131, 142]
[4, 130, 143]
[4, 130, 143]
[4, 129, 144]
[4, 128, 146]
[4, 128, 146]
[4, 128, 149]
[4, 128, 160]
[4, 130, 159]
[4, 148, 174]
[4, 147, 174]
[4, 178, 191]
[4, 174, 193]
[4, 335, 200]
[4, 335, 199]
[4, 346, 201]
[4, 338, 201]
[4, 339, 203]
[4, 332, 210]
[4, 331, 214]
[4, 331, 214]
[4, 325, 213]
[4, 306, 202]
[4, 307, 203]
[4, 387, 124]
[4, 390, 125]
[4, 394, 123]
[4, 390, 121]
[4, 388, 122]
[4, 382, 122]
[4, 379, 122]
[4, 368, 130]
[4, 364, 130]
[4, 359, 135]
[4, 358, 137]
[4, 362, 138]
[4, 362, 137]
[4, 369, 130]
[4, 373, 124]
[4, 376, 126]
[4, 388, 121]
[4, 393, 108]
[4, 392, 106]
[4, 396, 106]
[4, 388, 110]
[4, 387, 111]
[4, 386, 113]
[4, 381, 115]
[4, 378, 114]
[4, 378, 115]
[4, 380, 113]
[4, 383, 110]
[4, 385, 112]
[4, 385, 113]
[4, 385, 113]
[4, 38

In [7]:
#!/usr/bin/env python
# coding: utf-8

# In[1]:


import cv2
import mediapipe as mp
import time


# In[2]:


class HandDetector():
    def __init__(self, mode = False, maxHands = 2, detectionConf = 0.85, trackingConf = 0.85):
        self.mode = mode
        self.maxHands = maxHands
        self.detectionConf = detectionConf
        self.trackingConf = trackingConf

        self.mpHands = mp.solutions.hands
        self.hands = self.mpHands.Hands(static_image_mode = self.mode, 
                                        max_num_hands = self.maxHands,
                                        min_detection_confidence = self.detectionConf,
                                        min_tracking_confidence = self.trackingConf)
        
        self.mpDraw = mp.solutions.drawing_utils
        
    def findHands(self,img, draw = True):
        imgRGB = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgRGB)
        # print(results.multi_hand_landmarks)

        if self.results.multi_hand_landmarks:
            for handLms in self.results.multi_hand_landmarks:
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms, self.mpHands.HAND_CONNECTIONS)
        return img

    def findPosition(self, img, handNo = 0, draw = True):

        lmlist = []
        if self.results.multi_hand_landmarks:
            myHand = self.results.multi_hand_landmarks[handNo]
            for id, lm in enumerate(myHand.landmark):
                h, w, c = img.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                lmlist.append([id, cx, cy])
                if draw:
                    cv2.circle(img, (cx, cy), 7, (255, 0, 255), cv2.FILLED)
        return lmlist


# In[3]:


def main():
    pTime = 0
    cTime = 0
    cap = cv2.VideoCapture(0)
    detector = HandDetector()

    while True:
        success, img = cap.read()
        img = detector.findHands(img)
        lmlist = detector.findPosition(img)
        if len(lmlist) != 0:
            print(lmlist[4])

        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime

        cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3)

        cv2.imshow("Hand_Detector", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cv2.destroyWindow("Hand_Detector")


if __name__ == "__main__":
    main()

[4, 362, 259]
[4, 362, 260]
[4, 368, 257]
[4, 370, 257]
[4, 370, 255]
[4, 386, 254]
[4, 382, 253]
[4, 381, 247]
[4, 391, 251]
[4, 387, 250]
[4, 389, 246]
[4, 384, 248]
[4, 385, 248]
[4, 377, 251]
[4, 369, 255]
[4, 373, 259]
[4, 371, 258]
[4, 372, 258]
[4, 371, 258]
[4, 370, 256]
[4, 371, 257]
[4, 372, 257]
[4, 366, 260]
[4, 369, 261]
[4, 367, 263]
[4, 364, 266]
[4, 369, 263]
[4, 368, 264]
[4, 367, 263]
[4, 366, 264]
[4, 365, 265]
[4, 368, 264]
[4, 369, 263]
[4, 368, 264]
[4, 365, 267]
[4, 364, 267]
[4, 367, 264]
[4, 366, 264]
[4, 365, 266]
[4, 367, 265]
[4, 363, 266]
[4, 366, 266]
[4, 366, 266]
[4, 366, 267]
[4, 366, 267]
[4, 363, 266]
[4, 365, 267]
[4, 366, 266]
[4, 368, 266]
[4, 368, 266]
[4, 368, 260]
[4, 367, 261]
[4, 389, 239]


In [8]:
import HandAnchorCalucation
HandAnchorCalucation.main()

In [9]:
import PIL.Image
import Image
import ImageTk
from Tkinter import *    


class ExampleApp(Frame):
    def __init__(self,master):
        Frame.__init__(self,master=None)
        self.x = self.y = 0
        self.canvas = Canvas(self,  cursor="cross")

        self.sbarv=Scrollbar(self,orient=VERTICAL)
        self.sbarh=Scrollbar(self,orient=HORIZONTAL)
        self.sbarv.config(command=self.canvas.yview)
        self.sbarh.config(command=self.canvas.xview)

        self.canvas.config(yscrollcommand=self.sbarv.set)
        self.canvas.config(xscrollcommand=self.sbarh.set)

        self.canvas.grid(row=0,column=0,sticky=N+S+E+W)
        self.sbarv.grid(row=0,column=1,stick=N+S)
        self.sbarh.grid(row=1,column=0,sticky=E+W)

        self.canvas.bind("<ButtonPress-1>", self.on_button_press)
        self.canvas.bind("<B1-Motion>", self.on_move_press)
        self.canvas.bind("<ButtonRelease-1>", self.on_button_release)

        self.rect = None

        self.start_x = None
        self.start_y = None

        self.im = PIL.Image.open("logo.png")
        self.wazil,self.lard=self.im.size
        self.canvas.config(scrollregion=(0,0,self.wazil,self.lard))
        self.tk_im = ImageTk.PhotoImage(self.im)
        self.canvas.create_image(0,0,anchor="nw",image=self.tk_im)   


    def on_button_press(self, event):
        # save mouse drag start position
        self.start_x = self.canvas.canvasx(event.x)
        self.start_y = self.canvas.canvasy(event.y)

        # create rectangle if not yet exist
        if not self.rect:
            self.rect = self.canvas.create_rectangle(self.x, self.y, 1, 1, outline='red')

    def on_move_press(self, event):
        curX = self.canvas.canvasx(event.x)
        curY = self.canvas.canvasy(event.y)

        w, h = self.canvas.winfo_width(), self.canvas.winfo_height()
        if event.x > 0.9*w:
            self.canvas.xview_scroll(1, 'units') 
        elif event.x < 0.1*w:
            self.canvas.xview_scroll(-1, 'units')
        if event.y > 0.9*h:
            self.canvas.yview_scroll(1, 'units') 
        elif event.y < 0.1*h:
            self.canvas.yview_scroll(-1, 'units')

        # expand rectangle as you drag the mouse
        self.canvas.coords(self.rect, self.start_x, self.start_y, curX, curY)    

    def on_button_release(self, event):
        pass    

if __name__ == "__main__":
    root=Tk()
    app = ExampleApp(root)
    app.pack()
    root.mainloop()

ModuleNotFoundError: No module named 'Image'

In [19]:
import cv2
import numpy as np

def update_pts(params, x, y):
    global x_init, y_init
    params["top_left_pt"] = (min(x_init, x), min(y_init, y))
    params["bottom_right_pt"] = (max(x_init, x), max(y_init, y))
    img[y_init:y, x_init:x] = 255 - img[y_init:y, x_init:x]
    
def draw_rectangle(event, x, y, flags, params):
    global x_init, y_init, drawing
    # First click initialize the init rectangle point
    if event == cv2.EVENT_LBUTTONDOWN:
        drawing = True
        x_init, y_init = x, y
    # Meanwhile mouse button is pressed, update diagonal rectangle point
    elif event == cv2.EVENT_MOUSEMOVE and drawing:
        update_pts(params, x, y)
    # Once mouse botton is release
    elif event == cv2.EVENT_LBUTTONUP:
        drawing = False
        update_pts(params, x, y)
        
if __name__=='__main__':
    drawing = False
    event_params = {"top_left_pt": (-1, -1), "bottom_right_pt": (-1, -1)}
    
    cap = cv2.VideoCapture(0)

    # Check if the webcam is opened correctly
    if not cap.isOpened():
        raise IOError("Cannot open webcam")

    cv2.namedWindow('Webcam')
    # Bind draw_rectangle function to every mouse event
    cv2.setMouseCallback('Webcam', draw_rectangle, event_params)

    while True:
        ret, frame = cap.read()
        img = cv2.resize(frame, None, fx=0.5, fy=0.5,interpolation=cv2.INTER_AREA)
        (x0,y0), (x1,y1) = event_params["top_left_pt"], event_params["bottom_right_pt"]
        # img[y0:y1, x0:x1] = 255 - img[y0:y1, x0:x1]
        cv2.rectangle(frame, (x0, y0), (x1, y1), (255, 0, 255), 2)
        cv2.imshow('Webcam', frame)

        c = cv2.waitKey(1)
        if c == 27:
            break

    cap.release()
    cv2.destroyAllWindows()

In [20]:
#!/usr/bin/env python
# coding: utf-8

# In[13]:


import cv2
import mediapipe as mp
import time
import math


# In[14]:


class HandAnchorCalculation:  

    def __init__(self, mode = False, maxhands = 2, detectionConf = 0.85, trackingConf = 0.85):
        self.mode = mode
        self.maxhands = maxhands
        self.detectionConf = detectionConf
        self.trackingConf = trackingConf

        self.mphands = mp.solutions.hands
        self.hands = self.mphands.Hands(static_image_mode = self.mode, 
                                        max_num_hands = self.maxhands,
                                        min_detection_confidence = self.detectionConf,
                                        min_tracking_confidence = self.trackingConf)
        self.mpDraw = mp.solutions.drawing_utils
        
        self.tipIds = [4, 8, 12, 16, 20]
    
    
    def findHands(self, img, draw = True, fliptype = True):
        imgrgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        self.results = self.hands.process(imgrgb)
        allhands = []
        h, w, c = img.shape
        
        if self.results.multi_hand_landmarks:
            for handType,handLms in zip(self.results.multi_handedness, self.results.multi_hand_landmarks):
                handlist = {}
                lmlist = []
                xlist = []
                ylist = []
                
                for id, lm in enumerate(handLms.landmark):
                    px, py = int(lm.x * w), int(lm.y * h)
                    lmlist.append([px, py])
                    xlist.append(px)
                    ylist.append(py)
                
                # define hand box boundary
                xmin, xmax, ymin, ymax = min(xlist), max(xlist), min(ylist), max(ylist)
                bboxw, bboxh = xmax - xmin, ymax - ymin
                bbox = xmin, ymin, bboxw, bboxh
                cx, cy = xmin + (bboxw // 2), ymin + (bboxh // 2)
                
                # define the hand zone attribute
                handlist["lmlist"] = lmlist
                handlist["bbox"] = bbox
                handlist["center"] = (cx, cy)                
                if (fliptype == True):
                    if (handType.classification[0].label == "Right"):
                        handlist["type"] = "Left"
                    else:
                        handlist["type"] = "Right"
                else:
                    handlist["type"] = handType.classification[0].label
                    
                allhands.append(handlist)
                
                # draw the hand zone on frame
                if draw:
                    self.mpDraw.draw_landmarks(img, handLms, self.mphands.HAND_CONNECTIONS)
                    cv2.rectangle(img, (xmin-10, ymin-10), (xmax+10, ymax+10), (255, 0, 255), 2)
                    cv2.putText(img, handlist["type"], (xmin - 15, ymin - 15), cv2.FONT_HERSHEY_PLAIN, 2, (255, 0, 255), 2)
        
        if draw:
            return allhands, img
        else:
            return allhands
    
    
    def findFingersUp(self, handlist, img):
        fingers = []
        handtype = handlist["type"]
        lmlist = handlist["lmlist"]
        bbox = handlist["bbox"]
        
        if (self.results.multi_hand_landmarks):
            # Thumb
            if handtype == "Right":
                if lmlist[self.tipIds[0]][0] > lmlist[self.tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)
            else:
                if lmlist[self.tipIds[0]][0] < lmlist[self.tipIds[0] - 1][0]:
                    fingers.append(1)
                else:
                    fingers.append(0)

            # 4 Fingers
            for id in range(1, 5):
                if lmlist[self.tipIds[id]][1] < lmlist[self.tipIds[id] - 2][1]:
                    fingers.append(1)
                else:
                    fingers.append(0)                    
            
            cv2.putText(img, str(fingers) + "  " + str(sum(fingers)), (bbox[0] - 15, bbox[1] + bbox[3] + 35), cv2.FONT_HERSHEY_PLAIN, 1, (0, 100, 255), 1)
            
        return fingers
    
    def findHandDistance(self, p1, p2, img=None):
        x1, y1 = p1
        x2, y2 = p2
        cx, cy = (x1 + x2) // 2, (y1 + y2) // 2
        length = round(math.hypot(x2 - x1, y2 - y1), 2)
        info = (x1, y1, x2, y2, cx, cy)
        if img is not None:
            cv2.circle(img, (x1, y1), 9, (255, 0, 255), cv2.FILLED)
            cv2.circle(img, (x2, y2), 9, (255, 0, 255), cv2.FILLED)
            cv2.line(img, (x1, y1), (x2, y2), (255, 0, 255), 3)
            cv2.circle(img, (cx, cy), 9, (255, 0, 255), cv2.FILLED)
            
            cv2.putText(img, str(length), (cx, cy), cv2.FONT_HERSHEY_PLAIN, 2, (0, 255, 0), 2)
            return length,info, img
        else:
            return length, info
        
        
    def update_pts(self, params, x, y):
        global x_init, y_init
        params["top_left_pt"] = (min(x_init, x), min(y_init, y))
        params["bottom_right_pt"] = (max(x_init, x), max(y_init, y))
        img[y_init:y, x_init:x] = 255 - img[y_init:y, x_init:x]

    def draw_rectangle(self, event, x, y, flags, params):
        global x_init, y_init, drawing
        
        # First click initialize the init rectangle point
        if event == cv2.EVENT_LBUTTONDOWN:
            drawing = True
            x_init, y_init = x, y
            
        # Meanwhile mouse button is pressed, update diagonal rectangle point
        elif event == cv2.EVENT_MOUSEMOVE and drawing:
            update_pts(params, x, y)
            
        # Once mouse botton is release
        elif event == cv2.EVENT_LBUTTONUP:
            drawing = False
            update_pts(params, x, y)


# In[15]:


def main():
    pTime = 0
    cTime = 0
    
    drawing = False
    event_params = {"top_left_pt": (-1, -1), "bottom_right_pt": (-1, -1)}
    
    cap = cv2.VideoCapture(0)
    detector = HandAnchorCalculation()
    
    cv2.namedWindow('Webcam')
    # Bind draw_rectangle function to every mouse event
    cv2.setMouseCallback('Webcam', detector.draw_rectangle, event_params)

    while True:
        # Get image frame
        success, img = cap.read()
        # Find the hand and its landmarks with draw
        hands, img = detector.findHands(img)
        
        if hands:
            # first hand
            hand1 = hands[0]
            lmlist1 = hand1["lmlist"]  # List of 21 Landmark points
            bbox1 = hand1["bbox"]  # Bounding box info x,y,w,h
            centerPoint1 = hand1['center']  # center of the hand cx,cy
            handType1 = hand1["type"]  # Handtype Left or Right
            
            fingers1 = detector.findFingersUp(hand1, img)
            
            if len(hands) == 2:
                hand2 = hands[1]
                lmlist2 = hand2["lmlist"]  # List of 21 Landmark points
                bbox2 = hand2["bbox"]  # Bounding box info x,y,w,h
                centerPoint2 = hand2['center']  # center of the hand cx,cy
                handType2 = hand2["type"]  # Handtype Left or Right

                fingers2 = detector.findFingersUp(hand2, img)
                
                # Find Distance between two Landmarks. Could be same hand or different hands with draw
                length, info, img = detector.findHandDistance(lmlist1[8], lmlist2[8], img)
                
        cTime = time.time()
        fps = 1 / (cTime - pTime)
        pTime = cTime
        
        cv2.putText(img, str(int(fps)), (10, 70), cv2.FONT_HERSHEY_PLAIN, 3, (255, 0, 255), 3)
        
        (x0,y0), (x1,y1) = event_params["top_left_pt"], event_params["bottom_right_pt"]
        cv2.rectangle(img, (x0, y0), (x1, y1), (255, 0, 255), 2)
        
        cv2.imshow("Webcam", img)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyWindow("Webcam")
    
if __name__ == "__main__":
    main()


# In[ ]:






In [ ]:
import HAR
HAR.main()